In [84]:
import pandas as pd
import matplotlib.pyplot as plt

In [85]:
ty_df = pd.read_excel('../data/附件1.xlsx')
# sale_df = pd.read_excel('../data/附件2.xlsx')
purchase_df = pd.read_excel('../data/附件3.xlsx')
# decay_df = pd.read_excel('../data/附件4.xlsx')
ty_df.head(), purchase_df.head(), 
# sale_df.head(), decay_df.head()

(              单品编码    单品名称        分类编码 分类名称
 0  102900005115168    牛首生菜  1011010101  花叶类
 1  102900005115199   四川红香椿  1011010101  花叶类
 2  102900005115625  本地小毛白菜  1011010101  花叶类
 3  102900005115748     白菜苔  1011010101  花叶类
 4  102900005115762      苋菜  1011010101  花叶类,
           日期             单品编码  批发价格(元/千克)
 0 2020-07-01  102900005115762        3.88
 1 2020-07-01  102900005115779        6.72
 2 2020-07-01  102900005115786        3.19
 3 2020-07-01  102900005115793        9.24
 4 2020-07-01  102900005115823        7.03)

In [86]:
item_corpus = {row[1]['单品编码']: (row[1]['单品名称'], row[1]['分类编码']) 
               for row in ty_df.iterrows()}
ty_corpus = {row[1]['分类编码']: row[1]['分类名称'] 
             for row in ty_df.iterrows()}
print(list(item_corpus.items())[:5], list(ty_corpus.items())[:5], sep='\n')

[(102900005115168, ('牛首生菜', 1011010101)), (102900005115199, ('四川红香椿', 1011010101)), (102900005115625, ('本地小毛白菜', 1011010101)), (102900005115748, ('白菜苔', 1011010101)), (102900005115762, ('苋菜', 1011010101))]
[(1011010101, '花叶类'), (1011010201, '花菜类'), (1011010402, '水生根茎类'), (1011010501, '茄类'), (1011010504, '辣椒类')]


In [87]:
purchase_df['日期'] = pd.to_datetime(purchase_df['日期'])
purchase_df['年'] = purchase_df['日期'].dt.year
purchase_df['月'] = purchase_df['日期'].dt.month
purchase_df['日'] = purchase_df['日期'].dt.day
purchase_df.head()

,日期,单品编码,批发价格(元/千克),年,月,日
0,2020-07-01,102900005115762,3.88,2020,7,1
1,2020-07-01,102900005115779,6.72,2020,7,1
2,2020-07-01,102900005115786,3.19,2020,7,1
3,2020-07-01,102900005115793,9.24,2020,7,1
4,2020-07-01,102900005115823,7.03,2020,7,1


In [88]:
pch_grp = purchase_df.groupby(['单品编码'])
res = {}
month2season = {
    3: 0, 4: 0, 5: 0,
    6: 1, 7: 1, 8: 1,
    9: 2, 10: 2, 11: 2,
    12: 3, 1: 3, 2: 3
}
season_map = { 0: '春季', 1: '夏季', 2: '秋季', 3: '冬季' }
year_map = {2020: 0, 2021: 1, 2022: 2, 2023: 3}
rev_season_map = {v: k for k, v in season_map.items()}
for idx, grp in pch_grp:
    uni_months = grp['月'].unique()
    season_list = [0] * 4
    seasons = []
    season_cnt = []
    for month in uni_months:
        season = month2season[month]
        season_list[season] += 1
        seasons.append(season_map[season])
    num_seasons = 0
    for season in season_list:
        if season > 0:
            num_seasons += 1
    # 得出具体出产情况
    grp['季节'] = [season_map[month2season[month]] for month in list(grp['月'])]
    grp2 = grp.groupby(['季节', '年']).agg({'日期':'nunique'})
    record = [[0]*4 for _ in range(4)]
    for idx2, row2 in grp2.iterrows():
        # print(idx2, row2)
        season, year = idx2
        cnt = row2['日期']
        record[year_map[year]][rev_season_map[season]] = cnt
        # print(record)

    res[idx[0]] = {
        '月': sorted(uni_months),
        '月份数量': len(uni_months),
        '季节列表': season_list,
        '季节': seasons,
        '季节数量': num_seasons,
        '出产情况': record
    }
cnt = 0
for idx, row in res.items():
    print(f'{idx}: {row}')
    cnt += 1
    if cnt > 5:
        break

102900005115168: {'月': [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12], '月份数量': 11, '季节列表': [3, 3, 2, 3], '季节': ['秋季', '秋季', '冬季', '春季', '春季', '春季', '夏季', '夏季', '夏季', '冬季', '冬季'], '季节数量': 4, '出产情况': [[0, 0, 3, 1], [38, 14, 0, 31], [1, 3, 0, 17], [11, 0, 0, 1]]}
102900005115199: {'月': [2, 3, 4, 5, 11, 12], '月份数量': 6, '季节列表': [3, 0, 1, 2], '季节': ['秋季', '冬季', '冬季', '春季', '春季', '春季'], '季节数量': 3, '出产情况': [[0, 0, 12, 1], [65, 0, 3, 4], [43, 0, 0, 0], [37, 0, 0, 0]]}
102900005115250: {'月': [1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12], '月份数量': 11, '季节列表': [3, 2, 3, 3], '季节': ['秋季', '秋季', '冬季', '冬季', '冬季', '春季', '春季', '夏季', '春季', '夏季', '秋季'], '季节数量': 4, '出产情况': [[0, 0, 12, 2], [8, 2, 0, 39], [85, 24, 22, 7], [81, 30, 0, 14]]}
102900005115625: {'月': [9, 10, 11, 12], '月份数量': 4, '季节列表': [0, 0, 3, 1], '季节': ['秋季', '秋季', '秋季', '冬季'], '季节数量': 2, '出产情况': [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 46, 5], [0, 0, 0, 0]]}
102900005115748: {'月': [1, 2, 3, 5, 11, 12], '月份数量': 6, '季节列表': [2, 0, 1, 3], '季节': ['秋季', '冬季', '冬季', '春季', 

In [89]:
cnt_all = 0
cnt_all_list = []
for k, v in res.items():
    if v['季节数量'] == 4:
        cnt_all += 1
        cnt_all_list.append(k)
cnt_all, cnt_all_list[:5]

(112,
 [102900005115168,
  102900005115250,
  102900005115762,
  102900005115779,
  102900005115786])

In [90]:
result = purchase_df.groupby(['单品编码', '年']).agg({'日期': 'nunique'}).reset_index()
result.rename(columns={'日期': '天数'}, inplace=True)
result.head()

,单品编码,年,天数
0,102900005115168,2020,4
1,102900005115168,2021,83
2,102900005115168,2022,21
3,102900005115168,2023,12
4,102900005115199,2020,13


In [91]:
max_days = result.groupby('单品编码')['天数'].max().reset_index()
max_days.head()

,单品编码,天数
0,102900005115168,83
1,102900005115199,72
2,102900005115250,138
3,102900005115625,51
4,102900005115748,41


In [92]:
for idx, row in max_days.iterrows():
    index = row['单品编码']
    if row['天数'] > 300:
        # print(1)
        res[index]['性质'] = '常年性'
    elif row['天数'] < 15:
        # print(2)
        res[index]['性质'] = '时令性'
    else:
        # print(3)
        product = res[index]['出产情况']
        over_20_and_non_0 = []
        under_20 = []
        for year in product:
            cnt_0 = year.count(0)
            cnt_over = len([x for x in year if x > 20])
            cnt_under = len([x for x in year if x < 20])
            if cnt_0 == 0:
                over_20_and_non_0.append(cnt_over)
            elif cnt_under == 4:
                under_20.append(cnt_under)
        if len(over_20_and_non_0) == 4:
            res[index]['性质'] = '常年性'
        elif len(under_20) == 4:
            res[index]['性质'] = '时令性'
        else:
            res[index]['性质'] = '季节性'

In [93]:
cnt_list = [0] * 3
sample_list = [[]]*3
for idx, row in res.items():
    if row['性质'] == '常年性':
        cnt_list[0] += 1
        sample_list[0].append(item_corpus[idx][0])
    elif row['性质'] == '季节性':
        cnt_list[1] += 1
        sample_list[1].append(item_corpus[idx][0])
    elif row['性质'] == '时令性':
        cnt_list[2] += 1
        sample_list[2].append(item_corpus[idx][0])
print(cnt_list, sample_list)

[30, 136, 85] [['牛首生菜', '四川红香椿', '西峡花菇(1)', '本地小毛白菜', '白菜苔', '苋菜', '云南生菜', '竹叶菜', '小白菜', '南瓜尖', '上海青', '萝卜叶', '牛首油菜', '茼蒿', '蔡甸藜蒿', '菜心', '木耳菜', '大白菜', '豌豆尖', '云南油麦菜', '藕', '红尖椒', '青尖椒', '红椒(1)', '紫茄子(2)', '青茄子(1)', '西峡香菇(1)', '金针菇(1)', '马齿苋', '西兰花', '本地菠菜', '黄心菜(1)', '黑油菜', '平菇', '净藕(1)', '青杭椒(1)', '姬菇(1)', '红杭椒', '泡泡椒(精品)', '七彩椒(1)', '白玉菇(1)', '本地上海青', '菠菜', '高瓜(1)', '娃娃菜', '双孢菇', '蟹味菇(1)', '海鲜菇(1)', '青杭椒(2)', '红薯尖', '枝江红菜苔', '灯笼椒(1)', '小米椒', '杏鲍菇(1)', '黄花菜', '快菜', '螺丝椒', '紫圆茄', '随州泡泡青', '紫白菜(1)', '红线椒', '红灯笼椒(1)', '莲蓬(个)', '菱角', '姬菇(包)', '金针菇(袋)(1)', '田七', '东门口小白菜', '外地茼蒿', '冰草', '银耳(朵)', '紫苏', '薄荷叶', '蒲公英', '红莲藕带', '丝瓜尖', '奶白菜', '芥菜', '大芥兰', '面条菜', '甜白菜', '猴头菇', '荠菜', '鲜木耳(1)', '荸荠', '大龙茄子', '水果辣椒(橙色)', '青梗散花', '鸡枞菌', '野藕(1)', '黑牛肝菌', '海鲜菇(袋)(1)', '秀珍菇', '虫草花(盒)(1)', '茶树菇(袋)', '黑皮鸡枞菌', '杏鲍菇(袋)', '白玉菇(袋)', '马兰头', '甘蓝叶', '芜湖青椒(1)', '花茄子', '虫草花(袋)', '野生粉藕', '洪山菜苔', '赤松茸', '牛排菇', '洪湖莲藕(粉藕)', '组合椒系列', '长线茄', '本地黄心油菜', '青菜苔', '活体银耳', '小青菜(1)', '芜湖青椒(2)', '净藕(2)', '鲜藕带(袋)', '鲜粽子叶', '艾蒿', 